In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from numpy import array
from tensorflow.keras import layers


In [ ]:
# split a univariate sequence into samples
def split_sequence(sequence, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the sequence
		if out_end_ix > len(sequence):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix:out_end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

In [ ]:
d1 = np.genfromtxt('station_11.csv')
d2 = np.genfromtxt('station_32.csv')
d1=d1.reshape((d1.shape[0],1))
d2=d2.reshape((d2.shape[0],1))

In [ ]:
# choose a number of time steps
n_steps_in, n_steps_out = 24, 2
# split into samples
X, y = split_sequence(d2, n_steps_in, n_steps_out)
# summarize the data
n_features = 1
X = X.reshape((X.shape[0], X.shape[1], n_features))

In [ ]:
model = Sequential()
model.add(LSTM(100, activation='relu',  input_shape=(n_steps_in, n_features)))
model.add(Dense(n_steps_out))
model.compile(optimizer='adam', loss='mse')

In [ ]:
model.fit(X_train,y_train,epochs=50,verbose=0)

In [ ]:
pred = model.predict(X_test)

In [ ]:
for i in range(len(pred)):
  print(pred[i],y_test[i])

[-33.11078 193.24919] [[102.]
 [146.]]
[-41.63971 262.5089 ] [[69.]
 [94.]]
[146.7837  224.69193] [[243.]
 [179.]]
[ -9.152742 154.13326 ] [[174.]
 [185.]]
[115.05838 263.55334] [[146.]
 [146.]]
[129.7938  238.73192] [[134.]
 [ 85.]]
[156.22353 222.65361] [[213.]
 [176.]]
[130.334   143.88301] [[164.]
 [162.]]
[ 80.65343 177.26335] [[162.]
 [166.]]
[-41.645386 253.37737 ] [[65.]
 [69.]]
[-60.862473 220.69043 ] [[151.]
 [168.]]
[164.37535 219.73854] [[234.]
 [223.]]
[-71.87398 211.75595] [[105.]
 [112.]]


In [ ]:
# multivariate multi-step data preparation
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from sklearn.model_selection import train_test_split

# split a multivariate sequence into samples
def split_sequences(sequences, n_steps_in, n_steps_out):
	X, y = list(), list()
	for i in range(len(sequences)):
		# find the end of this pattern
		end_ix = i + n_steps_in
		out_end_ix = end_ix + n_steps_out
		# check if we are beyond the dataset
		if out_end_ix > len(sequences):
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequences[i:end_ix, :], sequences[end_ix:out_end_ix, :]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)

# horizontally stack columns
dataset = hstack((d1, d2))
# choose a number of time steps
n_steps_in, n_steps_out = 24, 2
# covert into input/output
X, y = split_sequences(dataset, n_steps_in, n_steps_out)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.06, random_state=42)

n_features = X.shape[2]
# define model
model = Sequential()
model.add(LSTM(200, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(200, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(X_train, y_train, epochs=300, verbose=0)


In [ ]:
ypred = model.predict(X_test)
ypred = ypred.reshape(-1,2)
y_test = y_test.reshape(-1,2)
from sklearn.metrics import mean_squared_error
print(mean_squared_error(y_test,ypred,squared=True))

25.732865451419954
